## Estimated strength and teams ranking

In this notebook we estimate the strength of handball teams and derive a ranking for European female clubs.

Based on [[1]](#references), the strength of a team is defined as

$$
s = s_a \cdot s_d = \dfrac{\log(\lambda_a) \cdot \nu_{d}}{\nu_{a} \cdot \log(\lambda_d)}.
$$

In [15]:
from handballPredictions import HandballData
import numpy as np

We first set some parameters to extract historical data.

In [2]:
connection = 'direct'
gender = 'F'
is_international = False

We fit the SEL parameters that are the strength and defense teams used for modeling scored goals.

In [10]:
hb_d = HandballData(gender=gender, international=is_international, connection=connection)
sel = hb_d.fit_sel_params(j=150)

Reading cached data


100%|██████████| 780/780 [26:30<00:00,  2.04s/it]


We then filter on female teams and leagues of interest.
We also extract historical data with scored goals.
This will be used to compute the empirical mean for each team.

In [11]:
# List leagues of interest
leagues = [k for k, v in hb_d.national_games.items() if v['gender'] == 'F']
# Load historical scores data
matches = hb_d._get_inputs()
teams = list(matches[(matches.league_id.isin(leagues)) & (matches.season == '2022/2023')].home_team_id.unique())

Reading cached data


In [12]:
def scored(x, season='2022/2023', o='home_score_final'):
    """Function to extract the empirical mean of scored goals per team and season
    """
    return matches[o][(matches.home_team_id == x) &( matches.season == season)].mean()

In [13]:
hb_d.sel_params['attack_emp'] = hb_d.sel_params['team_id'].apply(scored, o='home_score_final')
hb_d.sel_params['defense_emp'] = hb_d.sel_params['team_id'].apply(scored, o='away_score_final')
# We scale the defense strength by a factor of 10 to be in the same range as attack strength.
# By definition and construction of the strength, this does not affect the ranking.
hb_d.sel_params['defense_strength'] = 10* hb_d.sel_params['nu_defense'] / np.log(hb_d.sel_params['lambda_defense'])
# Strength = attack * defense
hb_d.sel_params['strength'] = hb_d.sel_params['attack_strength'] * hb_d.sel_params['defense_strength']

Let us display the top 10 strongest teams in Europe, estimated via our method.

In [19]:
_cols_interest = ['team_slug', 'attack_emp', 'defense_emp', 'attack_strength', 'defense_strength', 'strength']
ranking = hb_d.sel_params[_cols_interest][(hb_d.sel_params.season == '2022/2023') & (hb_d.sel_params.team_id.isin(teams))]
ranking.sort_values(by='strength', ascending=False).reset_index(drop=True).head(10)

,team_slug,attack_emp,defense_emp,attack_strength,defense_strength,strength
0,mks-zaglebie-lubin1,32.461538,22.153846,3.459735,3.203474,11.083171
1,vipers-kristiansand,36.956522,25.956522,3.562251,3.071824,10.942611
2,gyori-audi-eto-kc,32.956522,24.782609,3.482834,3.141009,10.939613
3,rk-podravka-vegeta,31.400000,22.000000,3.394068,3.202549,10.869670
4,metz-handball,33.190476,24.047619,3.478869,3.119963,10.853945
5,team-esbjerg,33.037037,24.296296,3.470639,3.113011,10.804136
6,sg-bbm-bietigheimw,34.750000,25.250000,3.547333,3.043750,10.797195
7,ik-savehof1,30.700000,24.750000,3.435066,3.114055,10.696983
8,dvsc-schaeffler,31.000000,24.578947,3.411876,3.117288,10.635801
9,hc-dac-dunajska-streda,28.666667,22.555556,3.365624,3.150181,10.602323


From the strongest teams of our ranking, we can observe that most are part of the European Champions League 2023/2024 and some (such as Vipers Kristiansand or Gyori Audi Eto KC) were part of the final four in edition 2022/2023.

## References

[1] Felice, F. (2023). Ranking Handball Teams from Statistical Strength Estimation. arXiv preprint arXiv:2307.06754